## Eigenvalues

Chapter 7.

The eigenvalue problem is to find all $\lambda$ and $v$ such that $Av=\lambda v$ for a matix $A$.

For hand calculations we usually find the roots of $\det(A-\lambda I)$. An $n$ x $n$ matrix has $n$ eigenvalues, counting double roots twice, etc. We can then form a diagonal matrix $D$ from the eigenvalues and a matrix $V$ with columns equal to the corresponding eigenvectors. Then we can write $AV = VD$. If $V$ is invertible, then $A = VDV^{-1}$. This is called the eigenvalue decomposition (EVD). If it exists, then $A$ is said to be diagonalizable.

If $A$ has $n$ distinct eigenvalues, then it is diagonalizable.

If $S$ is non-singular, then the similiarity transform $B = SAS^{-1}$ yields a matrix $B$ with the same eigenvalues as $A$.



Special terms for matrices with complex entries:

* A adjoint of a matrix $A$ is written $A^*$ and is computed as the complex conjugate of its transpose: $A^* = \bar{A}^T$. In Julia the transpose is computed using the single quotation operator: `A'` and for complex matrices, this is the conjugate transpose.
* A unitary matrix is a complex matrix with orthogonal columns, $A^* = A^{-1}$.

## Power method for finding algorithms

Section 8.2 detour.

Starting from an initial guess $v_0$ we can compute $v_{k+1} = Av_{k}$. If the dominant eigenvalue (largest magnitude) is 1, then this sequence will converge to the corresponding eigenvector. In all other cases, we normalize $v_k$ at each iteration to ensure its norm does not shrink to 0 or grow to infinity.

In [1]:
using LinearAlgebra
function my_power(A; v0 = randn(size(A)[1]), n = 100)
    v = v0
    last_lambda = Inf
    lambda = 1.0
    for i in 1:n
        v = A*v
        lambda = norm(v)
        v = v / lambda
        if abs(last_lambda - lambda ) < 1e-8
            break
        end
        last_lambda = lambda
    end
    (lambda, v)
end

my_power (generic function with 1 method)

In [2]:
A = [ 1 2 5 ; -2 4 0; 1 -1 1]

3×3 Matrix{Int64}:
  1   2  5
 -2   4  0
  1  -1  1

In [3]:
my_power(A, v0 = [1,1,1])

(3.675130889221877, [-0.15318478251892112, -0.9430553688467876, 0.29526427771384745])

In [4]:
my_power(A)

(3.675130888130893, [-0.15318478285523296, -0.9430553688206202, 0.29526427762294427])

In [5]:
eigen(A)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -0.21431974337753612
  2.5391888728108905
  3.6751308705666412
vectors:
3×3 Matrix{Float64}:
 -0.841424   0.584012   0.153185
 -0.399316   0.799573   0.943055
  0.364078  -0.140048  -0.295264

In [6]:
A = rand(3,3)
my_power(A, v0 = [1,1,1])

(2.098023299800106, [0.6287510540813102, 0.45881703462295226, 0.6278208667537595])

In [7]:
eigen(A)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -0.2541070272711071
 -0.050719488912378
  2.0980232999649915
vectors:
3×3 Matrix{Float64}:
  0.483171   0.805519  0.628751
 -0.865236  -0.372447  0.458817
  0.133839  -0.460892  0.627821

The convergence of this method depends on the ratio of the two largest eigenvalues.

The method can be adapted (Section 8.3, inverse iteration) to find the smallest eigenvalue.

Demo 7.2.12 shows how the QR factorization and iteration can be used to find eigenvalues and eigenvectors, but does not explain how the method works. See the Wikipedia article on the [Francis algorithm](https://en.wikipedia.org/wiki/QR_algorithm).

In [8]:
function my_qr_eigen(A0)
    A = copy(A0)
    for i in 1:40
      Q, R = qr(A)
      A = R*Q
    end
    sort(diag(A))
end

my_qr_eigen (generic function with 1 method)

The sequence $A_k$ of matrices are all similar (they have the same eigenvalues) since $A_{k+1} = R_kQ_k = Q_k^{-1}Q_kR_kQ_k = Q^{-1}_kAQ_k = Q^T_kAQ_k$

This sequence will (under some conditions) converge to the Shur form of $A$, which is upper triangular. The eigenvalues are then the diagonal entries.

In [9]:
my_qr_eigen(A)

3-element Vector{Float64}:
 -0.2541070272711068
 -0.05071948891237779
  2.0980232999649897

In [10]:
eigen(A)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -0.2541070272711071
 -0.050719488912378
  2.0980232999649915
vectors:
3×3 Matrix{Float64}:
  0.483171   0.805519  0.628751
 -0.865236  -0.372447  0.458817
  0.133839  -0.460892  0.627821

This implementation of the method does not find complex eigenvalues, so a better test is to generate a matrix with random real eigenvalues. First we compute a random orthogonal matrix and random eigenvalues, then use these data to create an eigenvalue problem.

In [11]:
N = 5
D = diagm( 4 .* randn(N) )
V,R = qr(randn(N,N))    # V is unitary
A = V*D*V'

5×5 Matrix{Float64}:
 -1.38806   0.822222  -1.34337   0.685584   0.696068
  0.822222  0.185604   1.13546   0.171165   0.513261
 -1.34337   1.13546    0.315399  0.524032  -1.03975
  0.685584  0.171165   0.524032  1.15788    0.452994
  0.696068  0.513261  -1.03975   0.452994  -1.03136

In [12]:
eigen(A)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
5-element Vector{Float64}:
 -2.8840013916607257
 -1.7718654278821173
  0.44377729202546506
  1.6523115335718743
  1.799238333351255
vectors:
5×5 Matrix{Float64}:
 -0.694332   0.476969   -0.249483   -0.472014  -0.073244
  0.403768   0.131573   -0.761381   -0.131771   0.4718
 -0.52948   -0.197257    0.0317123   0.456213   0.686735
  0.191163   0.0466413   0.548753   -0.607785   0.539211
 -0.194889  -0.845046   -0.236476   -0.426974  -0.0984245

In [13]:
D

5×5 Matrix{Float64}:
 -2.884  0.0      0.0       0.0       0.0
  0.0    1.79924  0.0       0.0       0.0
  0.0    0.0      0.443777  0.0       0.0
  0.0    0.0      0.0       1.65231   0.0
  0.0    0.0      0.0       0.0      -1.77187

In [14]:
my_qr_eigen(A)

5-element Vector{Float64}:
 -2.884001391660729
 -1.0031112178614159
  0.44377729202546523
  1.0487449550237447
  1.6340507018786818

A small extension will find the complex eigenvalues. Find each 2x2 block (with non-zero values off the diagonal) in the similar matrix $A_k$ and get each pair of complex conjugate eigenvalues from those blocks.

In [15]:
A = [ -1 2 3 ; 1 -1 2 ; 0 -1 2]

3×3 Matrix{Int64}:
 -1   2  3
  1  -1  2
  0  -1  2

In [16]:
eigen(A).values

3-element Vector{ComplexF64}:
 -2.425988757361625 + 0.0im
  1.212994378680811 - 1.189145108106551im
  1.212994378680811 + 1.189145108106551im

In [17]:
for i in 1:40
    Q,R = qr(A)
    A = R*Q
end
A

3×3 Matrix{Float64}:
 -2.42599     1.66448   0.886915
  1.8755e-6   0.879509  3.49309
  0.0        -0.436656  1.54648

In [18]:
eigen(A[2:3,2:3]).values

2-element Vector{ComplexF64}:
 1.21299393084036 - 1.1891450502365752im
 1.21299393084036 + 1.1891450502365752im

In [19]:
quad_roots(a,b,c) = (-b + sqrt(Complex(b^2 - 4*a*c)))/(2*a)
quad_roots(1, -A[2,2]-A[3,3], A[2,2]*A[3,3] - A[2,3]*A[3,2])

1.21299393084036 + 1.1891450502365757im

Ta Da!